In [28]:
import pandas as pd
import numpy as np

1 - Chargement du fichier & premières observations

In [29]:
df = pd.read_csv("./Data/events.csv", sep=',', encoding='utf-8')


In [30]:
df.head(10)

,timestamp,visitorid,event,itemid,transactionid
0,1433221332117,257597,view,355908,NaN
1,1433224214164,992329,view,248676,NaN
2,1433221999827,111016,view,318965,NaN
3,1433221955914,483717,view,253185,NaN
4,1433221337106,951259,view,367447,NaN
5,1433224086234,972639,view,22556,NaN
6,1433221923240,810725,view,443030,NaN
7,1433223291897,794181,view,439202,NaN
8,1433220899221,824915,view,428805,NaN
9,1433221204592,339335,view,82389,NaN


In [31]:
print(df.columns)
print(df.dtypes)
print("Total rows:", len(df))
print(df['event'].value_counts())
print(df['event'].value_counts(normalize=True))

Index(['timestamp', 'visitorid', 'event', 'itemid', 'transactionid'], dtype='object')
timestamp          int64
visitorid          int64
event             object
itemid             int64
transactionid    float64
dtype: object
Total rows: 2756101
event
view           2664312
addtocart        69332
transaction      22457
Name: count, dtype: int64
event
view           0.966696
addtocart      0.025156
transaction    0.008148
Name: proportion, dtype: float64


2 — Nettoyage & exploration 

In [ ]:
# Filtrer
df = df[df['event'].isin(['view','addtocart'])].copy()

# Vérifier valeurs manquantes
missing = df.isna().sum()
print("Missing:\n", missing)

# Nombre d'événements par type
print(df['event'].value_counts())

# Visiteurs uniques
n_visitors = df['visitorid'].nunique()
print("Visiteurs uniques:", n_visitors)


Missing:
 timestamp              0
visitorid              0
event                  0
itemid                 0
transactionid    2733644
dtype: int64
event
view         2664312
addtocart      69332
Name: count, dtype: int64
Visiteurs uniques: 1407500


3 — Simulation correcte d’un A/B test

In [ ]:
# Extraire visitors uniques
visitors = pd.DataFrame({'visitorid': df['visitorid'].unique()})
visitors.shape

# Assigner groupe A/B 50/50
rng = np.random.default_rng(seed=42)
perm = rng.permutation(len(visitors))
visitors['group'] = np.where(perm < len(visitors)/2, 'A', 'B')

# Joindre au dataset original (par visitorid)
df = df.merge(visitors, on='visitorid', how='left')

# Vérifier répartition visitors
print(visitors['group'].value_counts(normalize=True))

group
A    0.5
B    0.5
Name: proportion, dtype: float64


4 — Calcul du KPI : Add-to-Cart Rate

In [34]:
# Compter views et addtocart par groupe
agg = df.groupby(['group','event']).size().unstack(fill_value=0)
agg = agg.rename(columns={'view':'views','addtocart':'addtocarts'})
agg['addtocart_rate'] = agg['addtocarts'] / agg['views']
agg = agg.reset_index()
agg


event,group,addtocarts,views,addtocart_rate
0,A,34289,1333292,0.025718
1,B,35043,1331020,0.026328


5 — Test statistique : Test de proportion

Hypothèses :
- H0 : p_A = p_B
- H1 : p_A != p_B (test bilatéral)

Formules (manuel) :
- p̂_A = x_A / n_A (ici x = addtocarts, n = views)
- p̂_B = x_B / n_B
- p̂_pool = (x_A + x_B) / (n_A + n_B)
- z = (p̂_A - p̂_B) / sqrt( p̂_pool * (1 - p̂_pool) * (1/n_A + 1/n_B) )
- p-value (bilatéral) = 2 * (1 - Φ(|z|)) où Φ est la CDF normale standard.


In [35]:
import math

# Récupérer x_A, n_A, x_B, n_B
rowA = agg[agg['group']=='A'].iloc[0]
rowB = agg[agg['group']=='B'].iloc[0]

xA = int(rowA['addtocarts'])
nA = int(rowA['views'])
xB = int(rowB['addtocarts'])
nB = int(rowB['views'])

pA = xA / nA
pB = xB / nB
p_pool = (xA + xB) / (nA + nB)

se = math.sqrt(p_pool * (1 - p_pool) * (1/nA + 1/nB))
z = (pA - pB) / se

# Fonction Phi via erf
def phi(z):
    return 0.5*(1 + math.erf(z / math.sqrt(2)))

p_value = 2 * (1 - phi(abs(z)))

print(f"pA={pA:.6f}, pB={pB:.6f}, pooled={p_pool:.6f}")
print(f"z = {z:.4f}")
print(f"p-value (two-sided) = {p_value:.6f}")

pA=0.025718, pB=0.026328, pooled=0.026022
z = -3.1291
p-value (two-sided) = 0.001754


Différence entre les taux : B fait légèrement mieux que A (0.026328 vs 0.025718), mais l’écart est très petit.

Significativité : la p-value est 0.001754, donc la différence est statistiquement significative.

Risque d’erreur : faible (p-value très en dessous de 0.05), donc peu de chance que la différence soit due au hasard.

Décision :
    Si le coût de déploiement est faible → déployer B.
    Si le changement coûte cher pour un gain très faible → garder A.


In [36]:
import matplotlib.pyplot as plt

text = f"""
Contexte :
Nous avons réalisé un A/B testing avec les données RetailRocket (events.csv). 
L’idée était de comparer deux groupes d’utilisateurs (A et B) sur le taux d’ajout au panier. 
Nous avons gardé que les événements "view" et "addtocart". Chaque visiteur a été assigné 
aléatoirement à un groupe et cette assignation est restée fixe.

Méthodologie :
- Randomisation par visitorid (chaque utilisateur est dans un seul groupe)
- KPI : add-to-cart rate = addtocarts / views
- Test statistique : Z-test bilatéral avec α = 0.05
Nombre de visiteurs en A : {len(visitors[visitors['group']=='A'])}
Nombre de visiteurs en B : {len(visitors[visitors['group']=='B'])}

Résultats (KPI) :
Groupe A : {pA:.4%} ({xA}/{nA})
Groupe B : {pB:.4%} ({xB}/{nB})
Différence : {(pB - pA)*100:.3f} points de pourcentage (B est légèrement mieux)

Test statistique :
z = {z:.4f}
p-value = {p_value:.6f}
Seuil α = 0.05
→ La différence est statistiquement significative.

Décision :
B est un peu meilleur que A, mais l’écart est très faible. 
Donc soit on déploie B si c’est pas cher, soit on garde A si le gain est trop petit. 
Sinon, on peut essayer une variante C pour voir si on peut faire mieux.

Conclusion :
Avant de mettre en prod, il faut regarder si ce petit gain vaut vraiment le coup côté business.
"""

# Créer le PDF
fig = plt.figure(figsize=(8.27, 11.69)) 
plt.axis('off')
plt.text(0.01, 0.99, "TP4 — A/B Testing RetailRocket", fontsize=14, weight='bold', va='top')
plt.text(0.01, 0.92, text, fontsize=10, va='top')
plt.savefig("TP4_AB_Testing_report.pdf", bbox_inches='tight')
plt.close()

print("Report saved: TP4_AB_Testing_report.pdf")

Report saved: TP4_AB_Testing_report.pdf
